# Review Classification

## Batching and Model Training

### Import Libraries

In [1]:
import pandas as pd # Loading data
import numpy as np
import warnings 
import re # text matching
from collections import Counter # for vocabulary
from sklearn.model_selection import train_test_split # train test splits

warnings.filterwarnings('ignore')

### Data Loading and Processing

We will first do all the necessary pre-processing before starting to create batches and training the model. All the steps are explained in the notebook named `Text Cleaning.ipynb`

In [2]:
# Read dataset
data = pd.read_csv("Reviews.csv")
# Drop unnecesary columns and duplicates
new_data = data.drop_duplicates(subset=['UserId', 'ProfileName', 'Time', 'Text'])
# Get useful columns
useful_data = new_data[['Text', 'Score']]
# Calculate length of each sentence without tokenizer
useful_data['sudo_length'] = useful_data.Text.str.split().str.len()
# Filter examples by length
useful_data = useful_data[(useful_data.sudo_length > 20) & (useful_data.sudo_length < 100)]
# Remove length column
useful_data = useful_data.drop(['sudo_length'], axis = 1)
# print 5 rows
useful_data.head()

,Text,Score
0,I have bought several of the Vitality canned d...,5
1,Product arrived labeled as Jumbo Salted Peanut...,1
2,This is a confection that has been around a fe...,4
3,If you are looking for the secret ingredient i...,2
4,Great taffy at a great price. There was a wid...,5


### Tokenizing and Creating vocabulary
Now its time to tokenize and create our vocabulary. We use the `TextProcessor` class on data splits.

In [3]:
class TextProcessor:
    def __init__(self):
        self.vocab_dict = dict({"<unk>" : 0, "<pad>" : 1})
        self.counter = Counter()

    def tokenize(self, sent):
        if sent.endswith("."):
            sent = sent[:-1]
        new_x = re.sub('<.*?>', ' ', sent)
        new_x = re.sub('\s\s+',' ', new_x)
        new_x = re.sub('\W\s', ' ', new_x)
        new_x = re.sub('\w\W{2,}', ' ', new_x)
        new_x = new_x.lower().split()
        return new_x
        
    def processDataset(self, sent):
        tokens = self.tokenize(sent)
        token_set = set(tokens)
        self.counter.update(Counter(tokens))
        return len(tokens)
        
    def build_vocab(self, num_most_common_to_use=10000):
        words = self.counter.most_common(num_most_common_to_use)
        for i in range(num_most_common_to_use - 2):
            self.vocab_dict[words[i][0]] = len(self.vocab_dict)
            
    def tokenize_and_return_length(self, sent):
        tokens = self.tokenize(sent)
        return len(tokens)
            
    def process(self, sent):
        tokens = self.tokenize(sent)
        processed = []
        for val in tokens:
            processed.append(self.vocab_dict.get(val, self.vocab_dict["<unk>"]))
            
        return processed

#### Create Train and Test sets

In [4]:
train, test = train_test_split(useful_data, test_size = 0.2)

Run text processor to create vocabulary. Also create a new column denoting length of tokens for corresponding review. This will be used in creating batches.

In [5]:
textprocessor = TextProcessor()
train['length'] = train.Text.apply(textprocessor.processDataset)
textprocessor.build_vocab()

train.head()

,Text,Score,length
296098,I have tried a lot of tortellini and this is o...,5,59
239431,Not to say that I am a heavy martini drinker o...,4,59
552535,"I am a huge black licorice fan, however I don'...",5,48
2878,I've tried a handful of other brands of French...,5,37
21706,Our dog enjoys these better than any other har...,5,43


In [6]:
test['length'] = test.Text.apply(textprocessor.tokenize_and_return_length)

test.head()

,Text,Score,length
269963,Its simply water. No aftertaste. I tested it o...,5,64
560371,"What can I say, my wife has almost used up one...",5,29
503894,"This tastes even better than the ""Butter Chick...",5,33
127606,Lipton Family Size tea bags are a southern sta...,5,48
67146,These tomatoes are the best I have found for c...,5,36


### Batching and Data Loader creation
We are going to use `PyTorch` for training an `LSTM Model` for classification of reviews. Before creating the model, we first need to create dataloader, so that we can conviniently pass our training and testing examples to our model.

First we will create a *Custom PyTorch* dataset class which will preprocess our examples and convert them into a set of indices corresponging to vocabulary we just created.

In [7]:
import torch
from torch.utils.data import Dataset

In [8]:
class ReviewDataset(Dataset):
    def __init__(self, df, processor):
        self.data = df
        self.data = self.data.sort_values(by='length')
        self.tprocess = processor
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row.Text
        label = row.Score
        
        text = self.tprocess.process(text)
        
        return (text, label)

As you can observe that our dataset class sorted our dataframe by the length of each sentence. This allows us to create a batch with minmum padding, as we will see later when creating batches.

In [9]:
dataset = ReviewDataset(train, textprocessor)

print("Sample Data : ")
print(dataset[0])

Sample Data : 
([66, 0, 84, 0], 5)


### Creating Dataloader

#### Import Dataloader class

In [10]:
from torch.utils.data import DataLoader

#### Custom batch formation class
Since our dataset contains all the examples in sorted fashion, the batch we will get from our dataloader will have the largest length sentence at the end of the batch list. In the batch collator class, we will first create an array or size `(batch_size, seq_len)`, where seq_len will be equal to the length of last sentence recieved in batch.

As all the examples are sorted, padding required within a batch will be minimum as nearly equal length examples will be sampled.

In [11]:
class MyCollator(object):
    def __init__(self, pad_token = 1):
        self.pad = pad_token
    def __call__(self, batch):
        batch_size = len(batch)
        seq_len = len(batch[-1][0])
        formed = np.zeros((batch_size, seq_len), dtype = np.long) + self.pad
        labels = []
        for i in range(batch_size):
            example = batch[i]
            formed[i, :len(example[0])] = example[0]
            labels.append(example[1])
            
        return torch.LongTensor(formed), torch.LongTensor(labels)

In [12]:
collator = MyCollator()
dloader = DataLoader(dataset, batch_size=32, collate_fn=collator)

#### Example batch

In [13]:
batch = next(iter(dloader))
print("Examples : ")
print(batch[0])
print("Labels : ")
print(batch[1])

Examples : 
tensor([[  66,    0,   84,    0,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1],
        [   0,  111,  378,   10,   96,  226,   47, 1401,   63,  102,  654,    9,
            0,    1,    1,    1,    1,    1],
        [   0,  238,  150, 2100,   47,   36,  211,   51,    3,  198,   53,  567,
          257, 4796, 1624,    1,    1,    1],
        [5262,    0, 3065,    0, 1654, 8301, 2514,    0,    2,  185,  517,    7,
          768,  283,  201,    1,    1,    1],
        [ 118,  245,    0, 5873,  289,  366, 6545, 7169,   85,    0,  100,    5,
         3036,  207,    2, 1538,    1,    1],
        [ 608,    2,   62,   17, 4980,   29,  837,    2, 5837,   26,  229, 3609,
         3210,  129, 7900, 6453,    1,    1],
        [   8,  164,    9,  729, 3236,   14,    6,   65,  594,   56,  232,  371,
          172,    2, 6857, 4512,    1,    1],
        [  37,    8, 4990,   29,  985,   39,   65,  590, 1184,   14,  166, 1334,
           21,   2

### Calculating calss weights

In [14]:
from sklearn.utils.class_weight import compute_class_weight

weights = compute_class_weight('balanced', sorted(train.Score.unique()), train.Score)
for cat in sorted(train.Score.unique()):
    print("Class {} : {:.5f}".format(cat, weights[cat - 1]))

Class 1 : 2.16131
Class 2 : 3.99900
Class 3 : 2.94995
Class 4 : 1.54757
Class 5 : 0.30284


Now we are ready to start creating our model for Classification!!